In [6]:
# NOTEBOOK CONFIG

%idle_timeout 1440
%glue_version 4.0
%worker_type Standard
%number_of_workers 2

You are already connected to a glueetl session 41db922a-2d7f-4be5-b6cd-7a0e5cd69339.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Current idle_timeout is None minutes.
idle_timeout has been set to 1440 minutes.


You are already connected to a glueetl session 41db922a-2d7f-4be5-b6cd-7a0e5cd69339.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Setting Glue version to: 4.0


You are already connected to a glueetl session 41db922a-2d7f-4be5-b6cd-7a0e5cd69339.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous worker type: None
Setting new worker type to: Standard


You are already connected to a glueetl session 41db922a-2d7f-4be5-b6cd-7a0e5cd69339.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous number of workers: None
Setting new number of workers to: 2


In [96]:
###
seasons_to_update = ["046281"]
client_id =  8
# db_cluster_arn = "arn:aws:rds:eu-west-1:127023367472:cluster:traitsproddb"
# db_credentials_secret_store_arn = "arn:aws:secretsmanager:eu-west-1:127023367472:secret:traits-prod/aurora/mysql-aHJc2o"

In [97]:
### Libraries and config
import ast
import re
import sys
import json
import boto3
import pandas as pd
import pyspark.sql.functions as F
from datetime import datetime
from time import time
from pyspark.sql import SparkSession
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.dynamicframe import DynamicFrame
from awsglue.job import Job
from pyspark.sql.window import Window
from pyspark.sql.functions import explode, concat, col, lit, udf, expr, array_max, collect_list, sum as _sum, year, to_date, first, struct, input_file_name, regexp_extract, when, concat, countDistinct, rank
from pyspark.sql.types import IntegerType, StringType, ArrayType, StructType
from datetime import datetime, timedelta
from s3fs import S3FileSystem
import warnings

### Initialise clients
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
s3 = S3FileSystem()

In [98]:
# ### Variables ###

# # Retrieve arguments passed from STEP FUNCTION map as parameter
# # NB: Wyscout Data Warehouse is centralised, so client_id can be used as a proxy for production / other replication
# try:
#     args = getResolvedOptions(sys.argv, ['client_id', 'seasons_to_update'])
#     client_id = args['client_id']
# except:
#     args = getResolvedOptions(sys.argv, ['default_client_id', 'seasons_to_update'])
#     client_id = args['default_client_id']
    
# # Convert to strings
# seasons_to_update = [str(s) for s in ast.literal_eval(args['seasons_to_update'])]
    
# Threshold for valid games contributing to a profile sample
playtime_threshold = 33

In [99]:
# Columns to pass to output in addition to features
meta_cols = ['profileId',
'playerId',
'teamId',
'competitionId',
'seasonId',
'seasonPartition',
'startYear',
'endYear',
'age',
'playerName',
'teamName',
'seasonName',
'seasonDisplayName',
'competitionName',
'competitionShortName',
'fullName',
'nationality',
'birthDate',
'positionGroup',
'positionName',
'positionAbbreviation',
'teamSeason',
'playerTeamSeason',
'playerTeamSeasonCompetition',
]

integer_columns = ['startYear',
'endYear',
'age',
'totalMinutesInSample',
'sampleSize',
'totalMinutesForSeason',
'appearancesForSeason']

string_columns = ['profileId',
'playerId',
'teamId',
'competitionId',
'seasonId',
'seasonPartition',
'playerName',
'teamName',
'seasonName',
'seasonDisplayName',
'competitionName',
'competitionShortName',
'fullName',
'nationality',
'birthDate',
'positionGroup',
'positionName',
'positionAbbreviation',
'teamSeason',
'playerTeamSeason',
'playerTeamSeasonCompetition',
'aggregationPeriod']

In [100]:
### Helper functions ###

def flatten_struct(schema, prefix=None):
    """Generate a list of column expressions that flatten a DataFrame schema with prefixed nested column names."""
    field_exprs = []
    for field in schema.fields:
        name = field.name
        dtype = field.dataType
        current_col = f"`{name}`" if prefix is None else f"`{prefix}`.`{name}`"
        current_prefix = name if prefix is None else f"{prefix}_{name}"
        
        if isinstance(dtype, StructType):
            # If the field is a StructType, recurse to flatten it
            nested_exprs = flatten_struct(dtype, prefix=current_prefix)
            field_exprs.extend(nested_exprs)
        else:
            # For non-struct fields, alias the column with its prefixed name
            field_exprs.append(col(current_col).alias(current_prefix))
    return field_exprs

def calculate_age(birthDate, year):
    today = datetime.today()
    try:
        birthDate_obj = datetime.strptime(birthDate, '%Y-%m-%d')
        if today.year == year:
            age = today.year - birthDate_obj.year
            if today.month < birthDate_obj.month or (today.month == birthDate_obj.month and today.day < birthDate_obj.day):
                age -= 1
        else:
            age = int(year) - birthDate_obj.year
        return age
    except:
        return None
        
# def get_national_team(team_id, name):
#     return national_teams.get(str(team_id), name)

In [101]:
### Pre-load merge tables [retrieved information] ###

# Position mapping dictionary
try:
    with s3.open(f's3://traits-app/settings/positions_map.json', 'r', encoding='utf-8') as f:
        pos_key = json.load(f)['STATSBOMB']
    pos_key = {k: oldk for oldk, oldv in pos_key.items() for k in oldv}
except FileNotFoundError:
    print("positions_map.json file not found")
except ValueError:
    print("positions_map.json file is not in the expected format")
    
# Competitions [id, competition name]
comps_df = pd.read_json('s3://traits-app/settings/statsbomb_competitions.json', encoding='utf-8-sig')

# # National teams lookups
# with s3.open('s3://traits-app/bronze-raw/wyscout/national_teams.json', 'r', encoding='utf-8') as file:
#     national_teams = json.load(file)

# # Players
# player_path = f's3://traits-app/deployments/{client_id}/players.csv'
# player_df = pd.read_csv(player_path).drop_duplicates(subset=['offline_player_id'], keep='last')
# player_df['offline_player_id'] = player_df['offline_player_id'].astype('str')

In [102]:
### ETL PROCESSING ###    

print(f"Parsing seasons: {seasons_to_update}")

start = time()

# Read player statistics from json
player_s3_path = "s3://traits-app/bronze-raw/statsbomb/8/{" + ",".join(seasons_to_update) + "}/matches/*.json"
df_matches = spark.read.json(player_s3_path)

# Printing the "shape" of the DataFrame
def print_shape(df):
    print(f"Matches shape: Rows = {df_matches.count()}, Columns = {len(df_matches.columns)}")
    print("Games in selected seasons:", df_matches.select("match_id").distinct().count())
    
print_shape(df_matches)

# # Specify meta columns to include
# include_cols = ['seasonId', 'competitionId', 'roundId', 'matchId', 'playerId', 'positions',
#                 'player.birthDate', 'player.currentNationalTeamId', 'player.currentTeamId', 'player.shortName',
#                 'player.firstName', 'player.middleName', 'player.lastName', 'player.foot',
#                 'player.height', 'player.weight', 'player.gender',
#                 'player.imageDataURL', 'player.birthArea.name', 'player.role', 'player.status']

# # Generate the column expressions for the DataFrame
# flatten_exprs = [col(name) for name in include_cols] +\
#                 flatten_struct(df_matches.schema["total"].dataType, prefix="total") +\
#                 flatten_struct(df_matches.schema["average"].dataType, prefix="average") +\
#                 flatten_struct(df_matches.schema["percent"].dataType, prefix="percent")

# # Apply the expressions to select and alias the columns
# df_matches = df_matches.select(*flatten_exprs)

# Drop NA playerIds
# df_matches = df_matches.where(df_matches.player_id.isNotNull())

### Aggregate selected events into match level statistics
# NB: initially this is restricted to the statistics that appear in Statsbomb's season-level API

# Read in event derivations
event_derivations = pd.read_csv(f's3://traits-app/settings/stats_catalogue.csv').dropna(subset=['sql'])

# Start building the SQL query
sql_query_events = "SELECT match_id, player_id"

# Iterate through the DataFrame to populate the dictionary
for _, row in event_derivations.iterrows():
    stat_name = row['name_source']
    if pd.notna(row['sql']):  # Ensure there is a base SQL to work with
        sql_query_events += f", {row['sql']} AS {stat_name}"
    else:
        # raise
        print(ValueError(f"Valid base_sql does not exist for statistic '{stat_name}'"))

# Finish the SQL query by specifying the source and grouping condition
sql_query_events += " FROM {0} GROUP BY match_id, player_id"

# read events into spark, flatten, and derive x and y locations into 'location_x' and 'location_y'
events_s3_path = "s3://traits-app/bronze-raw/statsbomb/8/{" + ",".join(seasons_to_update) + "}/events/*.json"
df_events = spark.read.json(events_s3_path).select(
    #col("50_50.outcome.id").alias("50_50_outcome_id"),
    #col("50_50.outcome.name").alias("50_50_outcome_name"),
    #col("bad_behaviour.card.id").alias("bad_behaviour_card_id"),
    #col("bad_behaviour.card.name").alias("bad_behaviour_card_name"),
    col("ball_receipt.outcome.id").alias("ball_receipt_outcome_id"),
    col("ball_receipt.outcome.name").alias("ball_receipt_outcome"),
    col("ball_recovery.offensive").alias("ball_recovery_offensive"),
    col("ball_recovery.recovery_failure").alias("ball_recovery_recovery_failure"),
    col("block.deflection").alias("block_deflection"),
    col("block.offensive").alias("block_offensive"),
    col("block.save_block").alias("block_save_block"),
    col("carry.end_location").alias("carry_end_location"),
    #col("clearance.aerial_won").alias("clearance_aerial_won"),
    #col("clearance.body_part.id").alias("clearance_body_part_id"),
    #col("clearance.body_part.name").alias("clearance_body_part_name"),
    #col("clearance.head").alias("clearance_head"),
    #col("clearance.left_foot").alias("clearance_left_foot"),
    #col("clearance.other").alias("clearance_other"),
    #col("clearance.right_foot").alias("clearance_right_foot"),
    col("counterpress").alias("counterpress"),
    col("dribble.no_touch").alias("dribble_no_touch"),
    col("dribble.nutmeg").alias("dribble_nutmeg"),
    col("dribble.outcome.id").alias("dribble_outcome_id"),
    col("dribble.outcome.name").alias("dribble_outcome"),
    col("dribble.overrun").alias("dribble_overrun"),
    col("duel.outcome.id").alias("duel_outcome_id"),
    col("duel.outcome.name").alias("duel_outcome"),
    col("duel.type.id").alias("duel_type_id"),
    col("duel.type.name").alias("duel_type"),
    col("duration").alias("duration"),
    col("foul_committed.advantage").alias("foul_committed_advantage"),
    col("foul_committed.card.id").alias("foul_committed_card_id"),
    col("foul_committed.card.name").alias("foul_committed_card_name"),
    col("foul_committed.offensive").alias("foul_committed_offensive"),
    col("foul_committed.penalty").alias("foul_committed_penalty"),
    col("foul_committed.type.id").alias("foul_committed_type_id"),
    col("foul_committed.type.name").alias("foul_committed_type_name"),
    col("foul_won.advantage").alias("foul_won_advantage"),
    col("foul_won.defensive").alias("foul_won_defensive"),
    col("foul_won.penalty").alias("foul_won_penalty"),
    #col("goalkeeper.body_part.id").alias("goalkeeper_body_part_id"),
    #col("goalkeeper.body_part.name").alias("goalkeeper_body_part_name"),
    #col("goalkeeper.end_location").alias("goalkeeper_end_location"),
    #col("goalkeeper.lost_in_play").alias("goalkeeper_lost_in_play"),
    #col("goalkeeper.outcome.id").alias("goalkeeper_outcome_id"),
    #col("goalkeeper.outcome.name").alias("goalkeeper_outcome_name"),
    #col("goalkeeper.position.id").alias("goalkeeper_position_id"),
    #col("goalkeeper.position.name").alias("goalkeeper_position_name"),
    #col("goalkeeper.punched_out").alias("goalkeeper_punched_out"),
    #col("goalkeeper.shot_saved_off_target").alias("goalkeeper_shot_saved_off_target"),
    #col("goalkeeper.shot_saved_to_post").alias("goalkeeper_shot_saved_to_post"),
    #col("goalkeeper.technique.id").alias("goalkeeper_technique_id"),
    #col("goalkeeper.technique.name").alias("goalkeeper_technique_name"),
    #col("goalkeeper.type.id").alias("goalkeeper_type_id"),
    #col("goalkeeper.type.name").alias("goalkeeper_type_name"),
    #col("half_start.late_video_start").alias("half_start_late_video_start"),
    col("id").alias("id"),
    col("index").alias("index"),
    col("injury_stoppage.in_chain").alias("injury_stoppage_in_chain"),
    col("interception.outcome.id").alias("interception_outcome_id"),
    col("interception.outcome.name").alias("interception_outcome"),
    col("location").alias("location"),
    col("minute").alias("minute"),
    col("miscontrol.aerial_won").alias("miscontrol_aerial_won"),
    #col("obv_against_after").alias("obv_against_after"),
    #col("obv_against_before").alias("obv_against_before"),
    #col("obv_against_net").alias("obv_against_net"),
    #col("obv_for_after").alias("obv_for_after"),
    #col("obv_for_before").alias("obv_for_before"),
    #col("obv_for_net").alias("obv_for_net"),
    #col("obv_total_net").alias("obv_total_net"),
    #col("off_camera").alias("off_camera"),
    col("out").alias("out"),
    col("pass.aerial_won").alias("pass_aerial_won"),
    col("pass.angle").alias("pass_angle"),
    col("pass.assisted_shot_id").alias("pass_assisted_shot_id"),
    col("pass.body_part.id").alias("pass_body_part_id"),
    col("pass.body_part.name").alias("pass_body_part_name"),
    col("pass.cross").alias("pass_cross"),
    col("pass.cut_back").alias("pass_cut_back"),
    col("pass.deflected").alias("pass_deflected"),
    col("pass.end_location").alias("pass_end_location"),
    col("pass.goal_assist").alias("pass_goal_assist"),
    col("pass.height.id").alias("pass_height_id"),
    col("pass.height.name").alias("pass_height"),
    col("pass.inswinging").alias("pass_inswinging"),
    col("pass.length").alias("pass_length"),
    col("pass.miscommunication").alias("pass_miscommunication"),
    col("pass.no_touch").alias("pass_no_touch"),
    col("pass.outcome.id").alias("pass_outcome_id"),
    col("pass.outcome.name").alias("pass_outcome"),
    col("pass.outswinging").alias("pass_outswinging"),
    #col("pass.pass_cluster_id").alias("pass_pass_cluster_id"),
    #col("pass.pass_cluster_label").alias("pass_pass_cluster_label"),
    #col("pass.pass_cluster_probability").alias("pass_pass_cluster_probability"),
    #col("pass.pass_success_probability").alias("pass_pass_success_probability"),
    col("pass.recipient.id").alias("pass_recipient_id"),
    col("pass.recipient.name").alias("pass_recipient_name"),
    col("pass.shot_assist").alias("pass_shot_assist"),
    col("pass.straight").alias("pass_straight"),
    col("pass.switch").alias("pass_switch"),
    col("pass.technique.id").alias("pass_technique_id"),
    col("pass.technique.name").alias("pass_technique_name"),
    col("pass.through_ball").alias("pass_through_ball"),
    col("pass.type.id").alias("pass_type_id"),
    col("pass.type.name").alias("pass_type"),
    col("period").alias("period"),
    col("play_pattern.id").alias("play_pattern_id"),
    col("play_pattern.name").alias("play_pattern"),
    col("player.id").alias("player_id"),
    col("player.name").alias("player_name"),
    col("position.id").alias("position_id"),
    col("position.name").alias("position_name"),
    col("possession").alias("possession"),
    col("possession_team.id").alias("possession_team_id"),
    col("possession_team.name").alias("possession_team_name"),
    col("related_events").alias("related_events"),
    col("second").alias("second"),
    #col("shot.aerial_won").alias("shot_aerial_won"),
    #col("shot.body_part.id").alias("shot_body_part_id"),
    #col("shot.body_part.name").alias("shot_body_part_name"),
    #col("shot.deflected").alias("shot_deflected"),
    #col("shot.end_location").alias("shot_end_location"),
    #col("shot.first_time").alias("shot_first_time"),
    #col("shot.follows_dribble").alias("shot_follows_dribble"),
    #col("shot.gk_positioning_xg_suppression").alias("shot_gk_positioning_xg_suppression"),
    #col("shot.gk_save_difficulty_xg").alias("shot_gk_save_difficulty_xg"),
    #col("shot.gk_shot_stopping_xg_suppression").alias("shot_gk_shot_stopping_xg_suppression"),
    #col("shot.key_pass_id").alias("shot_key_pass_id"),
    # col("shot.one_on_one").alias("shot_one_on_one"),
    # col("shot.open_goal").alias("shot_open_goal"),
    # col("shot.outcome.id").alias("shot_outcome_id"),
    # col("shot.outcome.name").alias("shot_outcome"),
    # col("shot.redirect").alias("shot_redirect"),
    # col("shot.saved_off_target").alias("shot_saved_off_target"),
    # col("shot.saved_to_post").alias("shot_saved_to_post"),
    # col("shot.shot_execution_xg").alias("shot_shot_execution_xg"),
    # col("shot.shot_execution_xg_uplift").alias("shot_shot_execution_xg_uplift"),
    # col("shot.shot_shot_assist").alias("shot_shot_assist"),
    # col("shot.statsbomb_xg").alias("shot_statsbomb_xg"),
    # col("shot.technique.id").alias("shot_technique_id"),
    # col("shot.technique.name").alias("shot_technique_name"),
    # col("shot.type.id").alias("shot_type_id"),
    # col("shot.type.name").alias("shot_type_name"),
    # col("substitution.outcome.id").alias("substitution_outcome_id"),
    # col("substitution.outcome.name").alias("substitution_outcome"),
    # col("substitution.replacement.id").alias("substitution_replacement_id"),
    # col("substitution.replacement.name").alias("substitution_replacement_name"),
    # col("tactics.formation").alias("tactics_formation"),
    # col("tactics.lineup.jersey_number").alias("tactics_lineup_jersey_number"),
    # col("tactics.lineup.player.id").alias("tactics_lineup_player_id"),
    # col("tactics.lineup.player.name").alias("tactics_lineup_player_name"),
    # col("tactics.lineup.position.id").alias("tactics_lineup_position_id"),
    # col("tactics.lineup.position.name").alias("tactics_lineup_position_name"),
    col("team.id").alias("team_id"),
    col("team.name").alias("team_name"),
    col("timestamp").alias("timestamp"),
    col("type.id").alias("type_id"),
    col("type.name").alias("type"),
    col("under_pressure").alias("under_pressure")
).withColumn("location_x", col("location")[0]) \
             .withColumn("location_y", col("location")[1]) \
             .drop("location")

# Add match_id column
df_events = df_events.withColumn("filename", input_file_name())

# Parse match id
df_events = df_events.withColumn("match_id", regexp_extract("filename", r".*/(\d+).json", 1))

# Merge team possessions for adj stats derivations
df_events = df_events.join(df_matches.select(['player_id', 'match_id', 'player_match_possession']), ['player_id', 'match_id'])

# Create temp view
df_events.createOrReplaceTempView("events_matches")

# Apply the SQL statement
df_stats = spark.sql(sql_query_events.format("events_matches"))

# # TEMP: write stats data
# df_stats.toPandas().to_csv(f's3://traits-app/silver-cleaned/{client_id}/derived_stats_{competition_id}_{season_id}.csv')

# Merge derived stats onto matches dataframe
df_matches = df_matches.join(df_stats, ['match_id', 'player_id'])

print_shape(df_matches)

# Enforce dtypes on input frame
# for col_name in ['account_id', 'match_id', 'team_id', 'player_id', 'player_match_minutes', 'competition_id', 'season_id']:
#     df_matches = df_matches.withColumn(col_name, col(col_name).cast(IntegerType()))
# for col_name in ['team_name', 'player_name', 'competition_name', 'season_name']:
#     df_matches = df_matches.withColumn(col_name, col(col_name).cast(StringType()))
# # TODO: else float or double type

Parsing seasons: ['046281']
Matches shape: Rows = 9497, Columns = 156
Games in selected seasons: 312
Matches shape: Rows = 8263, Columns = 171
Games in selected seasons: 272


In [103]:
### PARSE LINEUPS FILES 
# 1. Cards information

# Read the JSON files for lineups
lineups_s3_path = "s3://traits-app/bronze-raw/statsbomb/8/{" + ",".join(seasons_to_update) + "}/lineups/*.json"
df_lineups = spark.read.json(lineups_s3_path)

# Add filename as column for match_id
df_lineups = df_lineups.withColumn("filename", input_file_name())

# Parse match id
df_lineups = df_lineups.withColumn("match_id", regexp_extract("filename", r".*/(\d+).json", 1))

# Further explode to get individual player data
df_exploded = df_lineups.selectExpr(
    "match_id",
    "team_name",
    "team_id",
    "explode(lineup) as player_data"
)

print("Games in selected_seasons (lineups endpoint):", df_lineups.select("match_id").distinct().count())

# Select the required fields
df_exploded = df_exploded.select(
    "match_id",
    "player_data.player_id", # merge col
    "player_data.player_name",
    "player_data.positions",
    "player_data.country",
    "player_data.player_nickname",
    "player_data.birth_date",
    "player_data.player_gender",
    "player_data.player_height",
    "player_data.player_weight",
    "player_data.jersey_number"
)

# Create default player column
df_exploded = df_exploded.withColumn("default_name", when(col("player_nickname").isNotNull(), col("player_nickname")).otherwise(col("player_name")))

# Parse the lineups df for yellow and red card events
df_lineup_events = df_lineups.select(
    "match_id",
    explode("events").alias("event")
)

# Filter events for "Yellow Card" and "Red Card"
df_filtered_events = df_lineup_events.filter(
    (col("event.outcome") == "Yellow Card") | (col("event.outcome") == "Red Card")
).select(
    "event.player_id",
    "match_id",
    "event.outcome"
)

# Count occurrences of each type of card per player
df_yellow_cards = df_filtered_events.filter(col("outcome") == "Yellow Card").groupBy("player_id", "match_id").count().withColumnRenamed("count", "yellow_cards")
df_red_cards = df_filtered_events.filter(col("outcome") == "Red Card").groupBy("player_id", "match_id").count().withColumnRenamed("count", "red_cards")

# Join the counts back to get a single row per player_id with counts for each card type
df_cards = df_yellow_cards.join(df_red_cards, ["player_id", "match_id"], "outer").fillna(0)

# Merge all the match by match lineup info
df_info = df_exploded.drop("player_name").join(df_cards.select(["player_id", "match_id", "red_cards", "yellow_cards"]), ["player_id", "match_id"], "left")

# Fill na in merge table
fill_values = {'player_height': 0, 'player_weight': 0, 'red_cards': 0, 'yellow_cards': 0}
df_info = df_info.fillna(fill_values)

# 2. Positions information

# Map the positionGroup to the Wyscout abbreviation the position map
broadcast_pos_map = spark.sparkContext.broadcast(pos_key)

def map_position_to_abbr(position_name):
    return broadcast_pos_map.value.get(position_name, None)

map_position_to_abbr_udf = udf(map_position_to_abbr, StringType())

# Extract the *first listed positionName and positionCode* from Wyscout's positions dictionary
# TODO: OPTIONAL: implement more advanced logic that extracts the highest minutes position, rather than the starting position
df_info = df_info.withColumn("primary_position", col("positions")[0])
df_info = df_info.withColumn("nationality", col("country.name"))
df_info = df_info.withColumn("positionName", col("primary_position.position"))
df_info = df_info.withColumn("positionAbbreviation", col("primary_position.position_id"))
df_info = df_info.withColumn("positionGroup", map_position_to_abbr_udf(col("positionName")))

# Show the original count in from the matches endpoint
print("Parse player match stats from lineups source:")
print_shape(df_info)
# SHOULD BE EQUAL TO df_matches shape if lineup data available

# Merge card and player info to match stats frame on player_id and match_id
df_matches_merged = df_matches.join(df_info, ["player_id", "match_id"], "left")

# TEMP: write match data
# df_matches_merged.toPandas().to_csv(f's3://traits-app/silver-cleaned/{client_id}/matches_data_{competition_id}_{season_id}.csv')

Games in selected_seasons (lineups endpoint): 273
Parse player match stats from lineups source:
Matches shape: Rows = 8263, Columns = 171
Games in selected seasons: 272


In [104]:
### Create "Any" designation

# Create rows that will not apply gametime thresholds, marked as "ANY" in positionGroup, positionName, positionCode
duplicated_df = df_matches_merged.withColumn("positionGroup", F.lit("ANY"))\
                .withColumn("positionName", F.lit("ANY"))\
                .withColumn("positionAbbreviation", F.lit("ANY"))

# Append to original positions frame
df_matches_merged = df_matches_merged.unionByName(duplicated_df)

# Drop rows where the total matchtime was under the threshold, but keep all "ANY" rows
# NB: this IGNORES that occasionally total game minutes will be over the threshold, but time spent in primary position will not be
df_split = df_matches_merged.filter(
    (col('player_match_minutes') > playtime_threshold) | (col('positionGroup') == "ANY")
)
print(f"Shape after dropping ineligible matches for all players frame: {df_split.count()}")

# ### OPTIONAL check for unique player per match rows

# # Check to ensure there is only two rows per player per match
# check_df = df_split.groupBy("playerId", "matchId").count()

# # # Filter to find invalid rows
# invalid_rows_df = check_df.filter(check_df['count'] > 2)

# # # Show the shape of the invalid filtered rows and a sample of those rows
# if invalid_rows_df.count() > 0:
#     print(f"Number of total rows (after union): {df_matches.count()}")
#     print(f"Number of invalid rows (after union): {invalid_rows_df.count()}")
#     # invalid_sample = invalid_rows_df.orderBy(F.col("playerId")).select("playerId", "matchId").show(5, truncate=False)
#     raise ValueError("There are more than two rows for some players in some matches (after union)")

# ###

Shape after dropping ineligible matches for all players frame: 14727


In [105]:
### ADD COMPETITION AND SEASON INFO
# TODO: work out how to pass comp id, season name, season id, then join on comps_df for display name
# TODO: season start and end are a nice to have too
competition_id = 47 #competition_season_id[0:3]
season_id = 281 #competition_season_id[3:]
competition_name = "JPL"
season_name = "2024"

# Append league and season ids as columns
df_split = df_split.withColumn("competition_id", lit(competition_id))\
            .withColumn("season_id", lit(season_id))\
            .withColumn("competition_name", lit(competition_name))\
            .withColumn("displayName", lit(competition_name))\
            .withColumn("season_name", lit(season_name))

In [106]:
### COLUMN CREATIONS TO CONFORM TO SILVER CONVENTION incl cAmelCasE
df_split = df_split.withColumnRenamed("player_id", "playerId")\
                        .withColumnRenamed("team_id", "teamId")\
                        .withColumnRenamed("season_id", "seasonId")\
                        .withColumnRenamed("competition_id", "competitionId")\
                        .withColumnRenamed("displayName", "competitionName")\
                         .withColumnRenamed("team_name", "teamName")\
                        .withColumnRenamed("season_name", "seasonDisplayName")\
                         .withColumnRenamed("default_name", "playerName")\
                         .withColumnRenamed("birth_date", "birthDate")\
                         .withColumnRenamed("player_weight", "playerWeight")\
                         .withColumnRenamed("player_height", "playerHeight")

                        #.withColumnRenamed("season.startDate", "seasonStart")\ # don't have
                        #.withColumnRenamed("season.endDate", "seasonEnd")\ # don't have
                        #.withColumnRenamed("nationality", "birthArea") # nationality parsed directly
            
### Create additional columns
# TODO: parse first and last four digits, confirm difference is one
df_split = df_split.withColumn("startYear", lit(2023)) #year(to_date(col("seasonStart"), "yyyy-MM-dd")))
df_split = df_split.withColumn("endYear", lit(2024)) #year(to_date(col("seasonEnd"), "yyyy-MM-dd")))
# TODO: derive seasonName from dates or naming convention udf (last 4 digit string)
# df_split = df_split.withColumn("seasonName", col("endYear"))
df_split = df_split.withColumn("seasonName", col("seasonDisplayName"))
df_split = df_split.withColumn("competitionShortName", col("seasonDisplayName")) # TODO: use comps_df Statsbomb name
df_split = df_split.withColumn("seasonPartition", col("seasonId"))
df_split = df_split.withColumn("fullName", col("player_name")) #F.concat_ws(" ", df_split["firstName"],  df_split["lastName"]))
df_split = df_split.withColumn("teamSeason", F.concat(df_split["teamName"], F.lit(" "), df_split["seasonName"]))
df_split = df_split.withColumn("playerTeamSeason", F.concat(df_split["playerName"], F.lit(" "), df_split["teamSeason"]))
df_split = df_split.withColumn("playerTeamSeasonCompetition", F.concat(df_split["playerTeamSeason"], F.lit(" "), df_split["competitionShortName"]))
df_split = df_split.withColumn("profileId", F.concat(df_split["playerId"].cast("string"), 
                                             df_split["teamId"].cast("string"),
                                             df_split["seasonId"].cast("string"), 
                                             df_split["competitionId"].cast("string"),
                                             df_split["positionGroup"]))

# # Apply nationality UDF # nationality parse as statsbomb 'country' designation
# get_national_team_udf = udf(get_national_team, StringType())
# df_split = df_split.withColumn("nationality", get_national_team_udf(col("currentNationalTeamId"), col("birthArea")))

# Apply age UDF
calculate_age_udf = udf(calculate_age, IntegerType())
df_split = df_split.withColumn("age", calculate_age_udf("birthDate", "endYear"))

In [107]:
# # OPTIONAL: (if running silver per client) read weights from S3
# try:
#     weights = pd.read_csv('s3://traits-app/settings/weights/{}.csv'.format(client_id))
# except FileNotFoundError:
#     print("weights csv file not found")
# except pd.errors.EmptyDataError:
#     print("weights csv is empty")

# required_features = weights['statName'].unique().tolist() 

# TODO: refactor below accordingly

# Read in feature formula map
retrieved_features = pd.read_csv(f's3://traits-app/settings/feature_store_statsbomb.csv')

# Start building the SQL query
sql_query_features = "SELECT m.profileId, COUNT(m.profileId) AS sampleSize, SUM(m.player_match_minutes) AS totalMinutesInSample"

# Iterate through the DataFrame to populate the dictionary
for _, row in retrieved_features.iterrows():
    feature_name = row['feature_name']
    if pd.notna(row['base_sql']):  # Ensure there is a base SQL to work with
        print(f"{feature_name} compiled.")
        sql_query_features += f", {row['base_sql']} AS {feature_name}"
    else:
        print(ValueError(f"Valid base_sql does not exist for feature '{feature_name}'"))
            
# Finish the SQL query by specifying the source and grouping condition
sql_query_features += " FROM {table_name} m GROUP BY m.profileId"

# Apply the SQL statement
df_split.createOrReplaceTempView("matches_season")
df_features = spark.sql(sql_query_features.format(table_name='matches_season'))

# Identify aggregationPeriod
df_features = df_features.withColumn("aggregationPeriod", lit("season"))

minutes compiled.
Valid base_sql does not exist for feature '90s_played'
aerial_ratio compiled.
aerial_wins_90 compiled.
aggressive_actions_90 compiled.
Valid base_sql does not exist for feature 'appearances'
assists_90 compiled.
Valid base_sql does not exist for feature 'op_assists_90'
Valid base_sql does not exist for feature 'average_minutes'
Valid base_sql does not exist for feature 'average_x_defensive_action'
average_x_pass compiled.
Valid base_sql does not exist for feature 'average_x_pressure'
backward_pass_proportion compiled.
ball_recoveries_90 compiled.
blocks_per_shot compiled.
box_cross_ratio compiled.
carries_90 compiled.
Valid base_sql does not exist for feature 'carry_length'
carry_ratio compiled.
challenge_ratio compiled.
Valid base_sql does not exist for feature 'change_in_passing_ratio'
Valid base_sql does not exist for feature 'clcaa'
clearance_90 compiled.
conversion_ratio compiled.
Valid base_sql does not exist for feature 'counterpressure_regains_90'
Valid base_s

In [108]:
### Repeat for last four and eight games
# TODO: wrap into function which accepts last_x parameter
# Filter for top 4 and top 8 matches (for each profile, not by gameweek)
windowSpec = Window.partitionBy("profileId").orderBy(col("match_id").desc()) # Assumes match ids are ascending
df_ranked = df_split.withColumn("rank", rank().over(windowSpec))
df_last4 = df_ranked.filter(col("rank") <= 4)
df_last8 = df_ranked.filter(col("rank") <= 8)

df_last4.createOrReplaceTempView("matches_4")
df_last8.createOrReplaceTempView("matches_8")

df_features_4 = spark.sql(sql_query_features.format(table_name="matches_4"))
df_features_8 = spark.sql(sql_query_features.format(table_name="matches_8"))

df_features_4 = df_features_4.withColumn("aggregationPeriod", lit("last_four"))
df_features_8 = df_features_8.withColumn("aggregationPeriod", lit("last_eight"))

# NB: pay attention to order of operations here
# Concatenate all the DataFrames
df_features = df_features.unionByName(df_features_4).unionByName(df_features_8)

In [109]:
### Remerge meta info to aggregated DataFrame
# NB: there may be duplicates on position_name here, hence information loss
df_features = df_features.join(df_split.select(meta_cols).dropDuplicates(['profileId']), on="profileId", how="left")

# NOTE: there will be duplicate profileIds at this point
# Update profileId (represents profile participation and aggregation)
df_features = df_features.withColumn("profileId", concat(df_features["profileId"], lit("-"), df_features["aggregationPeriod"]))

# Merge total player-season minutes and appearance counts to profiles
# NB: this does not apply any thresholding on contributing matches (see upstream)
any_df = df_features.filter((df_features.positionGroup == 'ANY') & (df_features.aggregationPeriod == 'season')) 
any_df = any_df.withColumnRenamed("totalMinutesInSample", "totalMinutesForSeason")\
                            .withColumnRenamed("sampleSize", "appearancesForSeason")

df_features = df_features.join(any_df.select("playerId", "teamId", "seasonId", "competitionId", "totalMinutesForSeason", "appearancesForSeason"),
                               on=["playerId", "teamId", "seasonId", "competitionId"], how="left") #  TODO: merge on pre-aggregation period profileId

print_shape(df_features)

# # TODO: Assert no duplicates
# assert df_features.count() - df_features.dropDuplicates(["playerTeamSeasonCompetition", "aggregationPeriod"]).count() == 0
# assert df_features.count() - df_features.dropDuplicates(["profileId"]).count() == 0

AssertionError: 


In [110]:
print_shape(df_features)

Matches shape: Rows = 8263, Columns = 171
Games in selected seasons: 272


In [111]:
print_shape(df_features.dropDuplicates(["playerTeamSeasonCompetition", "aggregationPeriod"]))

Matches shape: Rows = 8263, Columns = 171
Games in selected seasons: 272


In [112]:
print_shape(df_features.dropDuplicates(["profileId"]))

Matches shape: Rows = 8263, Columns = 171
Games in selected seasons: 272


In [113]:
### Enforce output table data types
# NB: all unspecified columns will be floats for simplicity

for column_name in df_features.columns:
    if column_name in integer_columns:
        df_features = df_features.withColumn(column_name, col(column_name).cast('integer'))
    elif column_name in string_columns:
        df_features = df_features.withColumn(column_name, col(column_name).cast('string'))
    else:
        df_features = df_features.withColumn(column_name, col(column_name).cast('double'))

In [114]:
# Basic write DataFrame to S3 with overwrite mode
df_features.write \
    .mode("overwrite") \
    .partitionBy("seasonPartition") \
    .format("parquet") \
    .option("compression", "snappy") \
    .save(f"s3://traits-app/silver-cleaned/{client_id}")

# TODO: add logs incl. schema pre-write, post-enforcement
df_features.printSchema()

print("Write successful.")

root
 |-- playerId: string (nullable = true)
 |-- teamId: string (nullable = true)
 |-- seasonId: string (nullable = true)
 |-- competitionId: string (nullable = true)
 |-- profileId: string (nullable = true)
 |-- sampleSize: integer (nullable = false)
 |-- totalMinutesInSample: integer (nullable = true)
 |-- minutes: double (nullable = true)
 |-- aerial_ratio: double (nullable = true)
 |-- aerial_wins_90: double (nullable = true)
 |-- aggressive_actions_90: double (nullable = true)
 |-- assists_90: double (nullable = true)
 |-- average_x_pass: double (nullable = true)
 |-- backward_pass_proportion: double (nullable = true)
 |-- ball_recoveries_90: double (nullable = true)
 |-- blocks_per_shot: double (nullable = true)
 |-- box_cross_ratio: double (nullable = true)
 |-- carries_90: double (nullable = true)
 |-- carry_ratio: double (nullable = false)
 |-- challenge_ratio: double (nullable = true)
 |-- clearance_90: double (nullable = true)
 |-- conversion_ratio: double (nullable = true)

In [115]:
### COMMENT OUT
df_features.toPandas().to_csv(f"s3://traits-app/silver-cleaned/{client_id}/statsbomb_features_test.csv")